# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

import random
import torch
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import argparse
import os
import utils
import TD3

import wandb

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="/home/corey/reinforcement-learning/udacity/deep-reinforcement-learning/p3_collab-compet/Tennis_Linux/Tennis.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.0


When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
env_info = env.reset(train_mode=True)[brain_name]

In [8]:

# Hyperparameters
policy ="TD3"                 # Policy name
seed = 0               # Sets Gym, PyTorch and Numpy seeds
start_timesteps = 25e3# Time steps initial random policy is used
max_timesteps = 329_329
expl_noise = 0.1              # Std of Gaussian exploration noise
batch_size = 256      # Batch size for both actor and critic
discount = 0.99                 # Discount factor
tau = 0.005                     # Target network update rate
policy_noise = 0.2              # Noise added to target policy during critic update
noise_clip = 0.5                # Range to clip target policy noise
policy_freq = 2       # Frequency of delayed policy updates
save_model = True        # Save model and optimizer parameters
load_model = ""                # Model load file name, "" doesn't load, "default" uses file_name

In [9]:

wandb.init(project="multi-agent")

wandb.config = {
    "policy": policy,
    "seed": seed, 
    #TODO add more project and run config
}

if not os.path.exists("./results"):
    os.makedirs("./results")

if save_model and not os.path.exists("./models"):
    os.makedirs("./models")
torch.manual_seed(seed)
np.random.seed(seed)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: coreymorris. Use `wandb login --relogin` to force relogin


In [10]:
env_info = env.reset(train_mode=True)[brain_name]

action_size = brain.vector_action_space_size

states = env_info.vector_observations
state_size = states.shape[1]

state_dim = state_size
action_dim = action_size
max_action = 1.0

In [11]:
kwargs = {
    "state_dim": state_dim,
    "action_dim": action_dim,
    "max_action": max_action,
    "discount": discount,
    "tau": tau,
}

# Initialize policy
if policy == "TD3":
    # Target policy smoothing is scaled wrt the action scale
    kwargs["policy_noise"] = policy_noise * max_action
    kwargs["noise_clip"] = noise_clip * max_action
    kwargs["policy_freq"] = policy_freq
    policy = TD3.TD3(**kwargs)
    policy_2 = TD3.TD3(**kwargs)

if load_model != "":
    policy_file = file_name if load_model == "default" else load_model
    policy.load(f"./models/{policy_file}")

replay_buffer = utils.ReplayBuffer(state_dim, action_dim)
replay_buffer_2 = utils.ReplayBuffer(state_dim, action_dim)

In [12]:
env_info = env.reset(train_mode=True)[brain_name] 
state = env_info.vector_observations[0]
done = env_info.local_done[0]

state_2 = env_info.vector_observations[1]
done_2 = env_info.local_done[1]
  
episode_reward = 0
episode_reward_2 = 0
episode_timesteps = 0
episode_num = 0

In [13]:

print_every = 100
scores_deque = deque(maxlen=print_every)
scores = []
for t in range(int(max_timesteps)):
    
    episode_timesteps += 1

    # Select action randomly or according to policy
    if t < start_timesteps:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)

    else:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        action_agent_zero = (
            policy.select_action(np.array(state))
            + np.random.normal(0, max_action * expl_noise, size=action_dim)
        ).clip(-max_action, max_action)

        action_agent_two = (
            policy_2.select_action(np.array(state_2))
            + np.random.normal(0, max_action * expl_noise, size=action_dim)
        ).clip(-max_action, max_action) 
        actions = [action_agent_zero, action_agent_two]

    # Perform action
    env_info = env.step(actions)[brain_name]
    next_state = env_info.vector_observations[0]
    next_state_2 = env_info.vector_observations[1]
    reward = env_info.rewards[0]
    reward_2 = env_info.rewards[1]
    done = env_info.local_done[0]
    done_2 = env_info.local_done[1]  
    done_bool = float(done) if env_info.max_reached[0] else 0
    done_bool_2 = float(done_2) if env_info.max_reached[1] else 0
    final_done = done_bool or done_bool_2

    # Store data in replay buffer
    replay_buffer.add(state, actions[0], next_state, reward, done_bool)
    replay_buffer_2.add(state_2, actions[1], next_state_2, reward_2, done_bool_2)

    state = next_state
    state_2 = next_state_2

    episode_reward += reward
    episode_reward_2 += reward_2

    # Train agent after collecting sufficient data
    if t >= start_timesteps:
        policy.train(replay_buffer, batch_size)
        policy_2.train(replay_buffer_2, batch_size)

    if (done or done_2): 
        # +1 to account for 0 indexing. +0 on ep_timesteps since it will increment +1 even if done=True
        # take the hightest episode reward as the reward reported for the episode as per the project instructions
        reported_episode_reward = max(episode_reward, episode_reward_2)
        wandb.log({
                    "episode number": episode_num + 1, 
                    "episode timesteps": episode_timesteps,
                    "episode reward": reported_episode_reward,
                    "total timesteps": t + 1})
        
        
        print(f"Total T: {t+1} Episode Num: {episode_num+1} Episode T: {episode_timesteps} Reward: {reported_episode_reward:.3f}")
        # Reset environment
        env_info = env.reset(train_mode=True)[brain_name] 
        state = env_info.vector_observations[0]
        state_2 = env_info.vector_observations[1]
        done = env_info.local_done[0]
        done_2 = env_info.local_done[1]
        scores.append(reported_episode_reward)
        scores_deque.append(reported_episode_reward)  
        episode_reward = 0
        episode_reward_2 = 0
        episode_timesteps = 0
        episode_num += 1
        if episode_num % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode_num, np.mean(scores_deque)))
            wandb.log({"Average Reward over last 100 episodes": np.mean(scores_deque)},commit=False)

wandb.finish()


Total T: 14 Episode Num: 1 Episode T: 14 Reward: 0.000
Total T: 28 Episode Num: 2 Episode T: 14 Reward: 0.000
Total T: 42 Episode Num: 3 Episode T: 14 Reward: 0.000
Total T: 56 Episode Num: 4 Episode T: 14 Reward: 0.000
Total T: 89 Episode Num: 5 Episode T: 33 Reward: 0.100
Total T: 144 Episode Num: 6 Episode T: 55 Reward: 0.100
Total T: 177 Episode Num: 7 Episode T: 33 Reward: 0.100
Total T: 191 Episode Num: 8 Episode T: 14 Reward: 0.000
Total T: 206 Episode Num: 9 Episode T: 15 Reward: 0.000
Total T: 220 Episode Num: 10 Episode T: 14 Reward: 0.000
Total T: 234 Episode Num: 11 Episode T: 14 Reward: 0.000
Total T: 250 Episode Num: 12 Episode T: 16 Reward: 0.000
Total T: 282 Episode Num: 13 Episode T: 32 Reward: 0.090
Total T: 296 Episode Num: 14 Episode T: 14 Reward: 0.000
Total T: 310 Episode Num: 15 Episode T: 14 Reward: 0.000
Total T: 324 Episode Num: 16 Episode T: 14 Reward: 0.000
Total T: 338 Episode Num: 17 Episode T: 14 Reward: 0.000
Total T: 352 Episode Num: 18 Episode T: 14 Re

Average Reward over last 100 episodes,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▄▇█
episode number,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
episode reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▄▄█▃
episode timesteps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▄█▂
total timesteps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▄▆█
Average Reward over last 100 episodes,2.385
episode number,3643
episode reward,2.7
episode timesteps,1001
total timesteps,328741


In [14]:
policy.save(f"./models/411_tennis_agent_1")

In [15]:
policy_2.save(f"./models/411_tennis_agent_2")